In [1]:
#예제 1
def deco(func):
  def inner():
    print('running inner()')
  return inner
@deco
def target():
  print("running target()")

target()

target

running inner()


<function __main__.deco.<locals>.inner()>

In [4]:
def logger(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG] {func.__name__} called with args={args},kwargs={kwargs} ")
    result = func(*args, **kwargs)
    print(f"[LOG] {func.__name__} returned {result}")
    return result
  return wrapper

@logger
def add(a,b):
  return a+b

add(5,12)

[LOG] add called with args=(5, 12),kwargs={} 
[LOG] add returned 17


17

In [5]:
#예제 2
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print("running main()")
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x7879bf28c9a0>)
running register(<function f2 at 0x7879bf28cae0>)
running main()
registry -> [<function f1 at 0x7879bf28c9a0>, <function f2 at 0x7879bf28cae0>]
running f1()
running f2()
running f3()


In [7]:
#예제 3
promos = []
def promotion(promo_func):
  promos.append(promo_func)
  return promo_func
@promotion
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item in order.cart:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  return max(promo(order) for promo in promos)

In [8]:
#예제 4
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [9]:
b =6
f1(3)

3
6


In [10]:
#예제 5
def f2(a):
  print(a)
  print(b)
  b=9
f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [11]:
#예제 6
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b=9
f3(3)

3
6


In [12]:
#예제 7
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self,new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)
avg = Averager()
avg(10)

10.0

In [13]:
avg(11)

10.5

In [14]:
avg(12)

11.0

In [17]:
#예제 8
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)
  return averager
avg = make_averager()
avg(10)

10.0

In [18]:
avg(11)

10.5

In [19]:
avg(12)

11.0

In [20]:
#예제 9
avg.__code__.co_varnames

('new_value', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

In [22]:
avg.__closure__

(<cell at 0x7879be06f400: list object at 0x7879be460dc0>,)

In [23]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [25]:
#예제 10
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    count+=1
    total += new_value
    return total / count

  return averager
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [27]:
#예제 11
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    nonlocal count, total
    count+=1
    total += new_value
    return total / count
  return averager
avg = make_averager()
avg(10)

10.0

In [29]:
#예제 12
import time
def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() -t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [31]:
#예제 13
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)
if __name__ == '__main__':
  print('*'*40, " calling snooze(.123)")
  snooze(.123)
  print('*'*40, "calling factorial(6)")
  print("6! = ", factorial(6))

****************************************  calling snooze(.123)
[0.12310552s] snooze(0.123) -> None
**************************************** calling factorial(6)
[0.00000053s] factorial(1) -> 1
[0.00002374s] factorial(2) -> 2
[0.00003447s] factorial(3) -> 6
[0.00004427s] factorial(4) -> 24
[0.00005356s] factorial(5) -> 120
[0.00006623s] factorial(6) -> 720
6! =  720


In [32]:
#예제 14
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args,**kwargs):
    t0 = time.perf_counter()
    result = func(*args,**kwargs)
    elapsed = time.perf_counter() -t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k,w) for k,w in sorted (kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [33]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n<2 else n*factorial(n-1)
if __name__ == '__main__':
  print('*'*40, " calling snooze(.123)")
  snooze(.123)
  print('*'*40, "calling factorial(6)")
  print("6! = ", factorial(6))

****************************************  calling snooze(.123)
[0.12316869s] snooze(0.123) -> None
**************************************** calling factorial(6)
[0.00000067s] factorial(1) -> 1
[0.00001453s] factorial(2) -> 2
[0.00002173s] factorial(3) -> 6
[0.00002963s] factorial(4) -> 24
[0.00003616s] factorial(5) -> 120
[0.00005213s] factorial(6) -> 720
6! =  720


In [34]:
factorial.__name__

'factorial'